In [1]:
import pandas as pd;
import numpy as np;
import os;

In [2]:
dfs = {i.split(".")[0]: pd.read_csv(f'data/{i}') for i in os.listdir('./data/')}

In [3]:
order_items = pd.read_csv("data/olist_order_items_dataset.csv")
orders = pd.read_csv("data/olist_orders_dataset.csv")
order_payments = pd.read_csv("data/olist_order_payments_dataset.csv")

In [4]:
print("\n**********\nOrder Items:\n",order_items.head())
print("\n**********\nOrders:\n",orders.head())
print("\n**********\nOrder Payments:\n",order_payments.head())


**********
Order Items:
                            order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00018f77f2f0320c557190d7a144bdd3              1   
2  000229ec398224ef6ca0657da4fc703e              1   
3  00024acbcdf0a6daa1e931b038114c75              1   
4  00042b26cf59d7ce69dfabb4e55b4fd9              1   

                         product_id                         seller_id  \
0  4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   
1  e5f2d52b802189ee658865ca93d83a8f  dd7ddc04e1b6c2c614352b383efe2d36   
2  c777355d18b72b67abbeef9df44fd0fd  5b51032eddd242adc84c38acab88f23d   
3  7634da152a4610f1595efa32f14722fc  9d7a1d34a5052409006425275ba1c2b4   
4  ac6c3623068f30de03045865e4e10089  df560393f3a51e74553ab94004ba5c87   

   shipping_limit_date   price  freight_value  
0  2017-09-19 09:45:35   58.90          13.29  
1  2017-05-03 11:05:13  239.90          19.93  
2  2018-01-18 14:48:30  199.00          17.87  
3  2018-08-15 10:1

In [5]:
#Build working dataframe
#Unique identifier of product is product_id-seller_id pair

#Convert timestamps to datetime
orders['order_purchase_timestamp']=pd.to_datetime(orders.order_purchase_timestamp)
orders['order_approved_at']=pd.to_datetime(orders.order_approved_at)
orders['order_delivered_carrier_date']=pd.to_datetime(orders.order_approved_at)
orders['order_delivered_customer_date']=pd.to_datetime(orders.order_delivered_customer_date)
orders['Time_to_customer'] = orders['order_delivered_customer_date']-orders['order_purchase_timestamp']

In [6]:
print(orders['Time_to_customer'])
orders['Time_to_customer']= orders['Time_to_customer'].dt.days

0        8 days 10:28:40
1       13 days 18:46:08
2        9 days 09:27:40
3       13 days 05:00:36
4        2 days 20:58:23
              ...       
99436    8 days 05:13:56
99437   22 days 04:38:58
99438   24 days 20:37:34
99439   17 days 02:04:27
99440    7 days 16:11:00
Name: Time_to_customer, Length: 99441, dtype: timedelta64[ns]


In [7]:
df = orders.merge(order_items, left_on="order_id",right_on="order_id",how="inner").drop(["order_approved_at","order_delivered_carrier_date","order_delivered_customer_date","order_estimated_delivery_date","freight_value","customer_id","shipping_limit_date","order_item_id","order_status","order_id"],axis=1)

In [8]:
df['date'] = df["order_purchase_timestamp"].dt.date

In [9]:
df = df.drop(['order_purchase_timestamp'],axis=1)
df

,Time_to_customer,product_id,seller_id,price,date
0,8.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,2017-10-02
1,13.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,118.70,2018-07-24
2,9.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,159.90,2018-08-08
3,13.0,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,45.00,2017-11-18
4,2.0,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,19.90,2018-02-13
...,...,...,...,...,...
112645,22.0,f1d4ce8c6dd66c47bbaa8c6781c2a923,1f9ab4708f3056ede07124aad39a2554,174.90,2018-02-06
112646,24.0,b80910977a37536adeddd63663f916ad,d50d79cb34e38265a8649c383dcffd48,205.99,2017-08-27
112647,17.0,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,179.99,2018-01-08
112648,17.0,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,179.99,2018-01-08


In [10]:
sales_quantity = df.groupby(["seller_id","product_id","date"]).count().reset_index()

In [11]:
sales_quantity['quantity'] = sales_quantity['price']
sales_quantity = sales_quantity.drop(["Time_to_customer","price"],axis=1)

In [12]:
sales_quantity

,seller_id,product_id,date,quantity
0,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,2017-09-26,1
1,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,2017-10-12,1
2,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,2017-10-18,1
3,001cca7ae9ae17fb1caed9dfb1094831,08574b074924071f4e201e151b152b4e,2017-03-15,2
4,001cca7ae9ae17fb1caed9dfb1094831,08574b074924071f4e201e151b152b4e,2017-03-25,1
...,...,...,...,...
94619,ffff564a4f9085cd26170f4732393726,dbd024d4182504993ad1e3cd2ee9d9e9,2017-01-16,1
94620,ffff564a4f9085cd26170f4732393726,dbd024d4182504993ad1e3cd2ee9d9e9,2017-01-19,1
94621,ffff564a4f9085cd26170f4732393726,de6517dda8e49774f58c07f80abc8d7a,2016-10-10,1
94622,ffff564a4f9085cd26170f4732393726,e20b58fe57d487f33247e6cc1154eb9c,2017-04-03,1


In [13]:
sales = df.merge(sales_quantity, left_on=["seller_id","product_id","date"],right_on=["seller_id","product_id","date"],how="inner")

In [14]:
sales = sales.drop_duplicates()
sales

,Time_to_customer,product_id,seller_id,price,date,quantity
0,8.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,2017-10-02,1
1,13.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,118.70,2018-07-24,1
2,9.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,159.90,2018-08-08,1
3,13.0,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,45.00,2017-11-18,1
4,2.0,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,19.90,2018-02-13,1
...,...,...,...,...,...,...
112645,11.0,ea73128566d1b082e5101ce46f8107c7,391fc6631aebcf3004804e51b40bcf1e,139.90,2017-02-23,1
112646,8.0,ac35486adb7b02598c182c2ff2e05254,e24fc9fcd865784fb25705606fe3dfe7,72.00,2017-03-09,1
112647,22.0,f1d4ce8c6dd66c47bbaa8c6781c2a923,1f9ab4708f3056ede07124aad39a2554,174.90,2018-02-06,1
112648,24.0,b80910977a37536adeddd63663f916ad,d50d79cb34e38265a8649c383dcffd48,205.99,2017-08-27,1


In [ ]:
#Threshold of at least 10 weeks of occurences. 
sales["Year"] = pd.to_datetime(sales['date']).dt.year
sales["Week_Num"] = pd.to_datetime(sales['date']).dt.week
sales["Year-Week"] = sales["Year"].astype(str)+"-"+sales["Week_Num"].astype(str)

In [16]:
print(sorted(sales["Year"].unique()),"\n")
print(sorted(sales["Week_Num"].unique()))
sales

[2016, 2017, 2018] 

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]


,Time_to_customer,product_id,seller_id,price,date,quantity,Year,Week_Num,Year-Week
0,8.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,2017-10-02,1,2017,40,2017-40
1,13.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,118.70,2018-07-24,1,2018,30,2018-30
2,9.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,159.90,2018-08-08,1,2018,32,2018-32
3,13.0,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,45.00,2017-11-18,1,2017,46,2017-46
4,2.0,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,19.90,2018-02-13,1,2018,7,2018-7
...,...,...,...,...,...,...,...,...,...
112645,11.0,ea73128566d1b082e5101ce46f8107c7,391fc6631aebcf3004804e51b40bcf1e,139.90,2017-02-23,1,2017,8,2017-8
112646,8.0,ac35486adb7b02598c182c2ff2e05254,e24fc9fcd865784fb25705606fe3dfe7,72.00,2017-03-09,1,2017,10,2017-10
112647,22.0,f1d4ce8c6dd66c47bbaa8c6781c2a923,1f9ab4708f3056ede07124aad39a2554,174.90,2018-02-06,1,2018,6,2018-6
112648,24.0,b80910977a37536adeddd63663f916ad,d50d79cb34e38265a8649c383dcffd48,205.99,2017-08-27,1,2017,34,2017-34


In [17]:
yw_table = sales[['product_id','seller_id','Year-Week']].drop_duplicates()

In [18]:
yw_table['count'] = 0
occurences = yw_table.groupby(['product_id','seller_id']).count().reset_index().drop(['Year-Week'],axis=1)
occurences[occurences['count']>=8].drop_duplicates()

,product_id,seller_id,count
9,001b72dfd63e9833e8c02742adf472e3,8a32e327fe2c1b3511609d81aaf9f042,10
12,002159fe700ed3521f46cfcf6e941c76,048c2757535328e0d7dac690ad3c0aae,8
38,005030ef108f58b46b78116f754d8d38,3c7c4a49ec3c6550809089c6a2ca9370,10
58,008cff0e5792219fae03e570f980b330,530ec6109d11eaaf87999465c6afee01,16
77,00ba6d766f0b1d7b78a5ce3e1e033263,53e4c6e0f4312d4d2107a8c9cddf45cd,10
...,...,...,...
34306,ff29d8cb1cd0cd5ea37b80dac9939e1c,8b321bb669392f5163d04c59e235e066,14
34329,ff55834c0b3272853dfe62340f2bb91b,6560211a19b47992c3666cc44a7e94c0,8
34353,ff7fccf8513f360157f0660fe51d1d88,440dd6ab244315c632130ecfb63827b1,11
34412,ffc9caf33e2d1e9f44e3e06da19085f7,b18dc380845b24038cfc48006478f099,9


In [19]:
occurences =occurences[occurences['count']>=8]
sales = sales.merge(occurences, left_on=["product_id","seller_id"],right_on=["product_id","seller_id"],how="inner").drop(["count"],axis=1)

### Final Sales dataset containing items that have 8 or more weeks of occurences in the dataset

In [20]:
sales

,Time_to_customer,product_id,seller_id,price,date,quantity,Year,Week_Num,Year-Week
0,13.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,118.7,2018-07-24,1,2018,30,2018-30
1,5.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,116.9,2018-08-09,1,2018,32,2018-32
2,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,116.9,2018-08-12,2,2018,32,2018-32
3,4.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,116.9,2018-08-12,2,2018,32,2018-32
4,2.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,118.7,2018-07-09,1,2018,28,2018-28
...,...,...,...,...,...,...,...,...,...
39204,10.0,00faa46f36261af8bbf3a4d37fa4841b,b372ee768ed69e46ca8cdbd267aa7a38,270.0,2018-02-06,1,2018,6,2018-6
39205,13.0,00faa46f36261af8bbf3a4d37fa4841b,b372ee768ed69e46ca8cdbd267aa7a38,270.0,2017-11-13,1,2017,46,2017-46
39206,13.0,00faa46f36261af8bbf3a4d37fa4841b,b372ee768ed69e46ca8cdbd267aa7a38,270.0,2017-12-05,1,2017,49,2017-49
39207,24.0,00faa46f36261af8bbf3a4d37fa4841b,b372ee768ed69e46ca8cdbd267aa7a38,295.0,2018-05-19,1,2018,20,2018-20
